In [1]:
import os
import tqdm
import pandas as pd
import numpy as np
from pathlib import Path

import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2analyzer as bwa
import time

c:\Users\maxime.pollet\Documents\Code\venv\simbat\Lib\site-packages\bw2calc\__init__.py:57: UserWarning: No fast sparse solver found
  warnings.warn("No fast sparse solver found")


In [2]:
bd.projects.set_current('acv_structures')

In [3]:
bd.databases

Databases dictionary with 10 object(s):
	acier
	biosphere3
	bois
	ciment_beton
	ecoinvent-3.9.1-cutoff
	housing_structures
	plancher
	plancher_poutre
	poteau
	poutre

In [4]:
housing_db = bd.Database('housing_structures')
eidb = bd.Database('ecoinvent-3.9.1-cutoff')

In [5]:
# Collect relevant processes
# timber
process_glt = [
    act for act in bd.Database("bois") \
        if 'glued laminated timber, A-C'== act['name']
][0]
process_clt = [
    act for act in bd.Database("bois") \
        if 'cross laminated timber, A-C' == act['name']
][0]
process_wood_joist = [
    act for act in bd.Database("bois") \
        if 'wood joist, A-C' == act['name']
][0]
# Steel
process_reinforcing_steel = [
    act for act in bd.Database("acier") \
        if 'reinforcing steel, A-C' == act['name']
][0]
process_sheet_steel = [
    act for act in bd.Database("acier") \
        if 'sheet steel, A-C' == act['name']
][0]
process_standard_steel = [
    act for act in bd.Database("acier") \
        if 'standard steel, A-C' == act['name']
][0]
process_plaster = [
    act for act in bd.Database("acier") if 'plaster' == act['name']
][0]
process_stone_wool = [
    act for act in bd.Database("acier") if 'stone wool' == act['name']
][0]
# Concrete
process_concrete_standard = [
    act for act in bd.Database("ciment_beton") \
        if 'concrete, cement CEM II/A, building construction, A-C,' \
            + ' economic allocation' == act['name']
][0]
process_concrete_screed = [
    act for act in bd.Database("ciment_beton") \
        if 'concrete, cement CEM II/A, for lean concrete, A-C,' \
            + ' economic allocation' == act['name']
][0]
process_concrete_fondation = [
    act for act in bd.Database("ciment_beton") if \
        'concrete, cement CEM II/A, civil engineering, A-C,' \
            + ' economic allocation' == act['name']][0]

process_hvac = [
    a for a in eidb.search('heat production, air-water heat pump') \
        if 'Europe' in a['location']
][0]

# Store material processes functional unit conversion coeffs
# (to translate kg to m3 when needed)
# Get weights of for 1 m3 of each material
p_glt = 490
p_clt = 480
p_bois_massif = 480
p_cemii = 2400
p_osb = 616
p_acier = 7850
processes = {
    'standard_steel': {
        'names': ['Acier-S235'],
        'process': process_standard_steel,
        'FU_conversion_coeff': 1
    },
    'glt': {
        'names': ['Bois-GL24h'],
        'process': process_glt,
        'FU_conversion_coeff': 1/p_glt
    },
    'standard_concrete': {
        'names': ['Béton CEMII', 'Dalle Béton'],
        'process': process_concrete_standard,
        'FU_conversion_coeff': 1/p_cemii
    },
    'reinforcing_steel': {
        'names': ['Ferraillage'],
        'process': process_reinforcing_steel,
        'FU_conversion_coeff': 1
    },
    'sheet_steel': {
        'names': ['Acier-Assemblage', 'Tôle'],
        'process': process_sheet_steel,
        'FU_conversion_coeff': 1
    },
    'concrete_screed': {
        'names': ['Chape Béton'],
        'process': process_concrete_screed,
        'FU_conversion_coeff': 1/p_cemii
    },
    'clt': {
        'names': ['-CLT'],
        'process': process_clt,
        'FU_conversion_coeff': 1/p_clt
    },
    'wood_joist': {
        'names': ['Bois-C18', 'Solives bois'],
        'process': process_wood_joist,
        'FU_conversion_coeff': 1/p_bois_massif
    },
    'plaster': {
        'names': ['Platre'],
        'process': process_plaster,
        'FU_conversion_coeff': 1
    },
    'concrete_foundation': {
        'names': ['Fondation-Beton'],
        'process': process_concrete_fondation,
        'FU_conversion_coeff': 1/p_cemii
    },
    'stone_wool': {
        'names': ['Flocage'],
        'process': process_stone_wool,
        'FU_conversion_coeff': 1
    },
    'hvac': {
        'names': ['Total HVAC consumption (kWh/year)'],
        'process': process_hvac,
        'FU_conversion_coeff': 3.6
    }
}

In [6]:
# Select methods
recipe_methods = [
    m for m in bd.methods if 'recipe 2016 v1.03' in m[0].lower()
    and '(h)' in m[0].lower() and 'no lt' not in m[0].lower()
]

# Perform LCA
functional_units = {}
for process_dict in processes.values():
    functional_units[process_dict['process']['name']] = {
        process_dict['process'].id: 1
    }
config = {
    'impact_categories': recipe_methods
}
data_objs = bd.get_multilca_data_objs(
    functional_units=functional_units,
    method_config=config
)
mlca = bc.MultiLCA(
    demands=functional_units,
    method_config=config,
    data_objs=data_objs,
    use_distributions=True
)
mlca.lci()
mlca.lcia()
scores = [mlca.scores for _ in tqdm.tqdm(zip(mlca, range(10000)), total=10000)]

100%|██████████| 10000/10000 [6:53:31<00:00,  2.48s/it] 


In [7]:
# Instantiate results dict
results_dict = {}
# Iterate over material
for mat in functional_units.keys():
    # Iterate over methods
    for m in recipe_methods:
        results_dict[f'{mat}_{m}'] = []

# Iterate over scores and save results
for res in scores:
    for key, val in res.items():
        results_dict[f'{key[-1]}_{key[0]}'].append(val)

# Compile results in a df and save them to csv
df = pd.DataFrame(results_dict)
df.index.name = 'Run ID'
df

,"standard steel, A-C_('ReCiPe 2016 v1.03, endpoint (H)', 'ecosystem quality', 'acidification: terrestrial')","standard steel, A-C_('ReCiPe 2016 v1.03, endpoint (H)', 'ecosystem quality', 'climate change: freshwater ecosystems')","standard steel, A-C_('ReCiPe 2016 v1.03, endpoint (H)', 'ecosystem quality', 'climate change: terrestrial ecosystems')","standard steel, A-C_('ReCiPe 2016 v1.03, endpoint (H)', 'ecosystem quality', 'ecotoxicity: freshwater')","standard steel, A-C_('ReCiPe 2016 v1.03, endpoint (H)', 'ecosystem quality', 'ecotoxicity: marine')","standard steel, A-C_('ReCiPe 2016 v1.03, endpoint (H)', 'ecosystem quality', 'ecotoxicity: terrestrial')","standard steel, A-C_('ReCiPe 2016 v1.03, endpoint (H)', 'ecosystem quality', 'eutrophication: freshwater')","standard steel, A-C_('ReCiPe 2016 v1.03, endpoint (H)', 'ecosystem quality', 'eutrophication: marine')","standard steel, A-C_('ReCiPe 2016 v1.03, endpoint (H)', 'ecosystem quality', 'land use')","standard steel, A-C_('ReCiPe 2016 v1.03, endpoint (H)', 'ecosystem quality', 'photochemical oxidant formation: terrestrial ecosystems')",...,"heat production, air-water heat pump 10kW_('ReCiPe 2016 v1.03, midpoint (H)', 'human toxicity: carcinogenic', 'human toxicity potential (HTPc)')","heat production, air-water heat pump 10kW_('ReCiPe 2016 v1.03, midpoint (H)', 'human toxicity: non-carcinogenic', 'human toxicity potential (HTPnc)')","heat production, air-water heat pump 10kW_('ReCiPe 2016 v1.03, midpoint (H)', 'ionising radiation', 'ionising radiation potential (IRP)')","heat production, air-water heat pump 10kW_('ReCiPe 2016 v1.03, midpoint (H)', 'land use', 'agricultural land occupation (LOP)')","heat production, air-water heat pump 10kW_('ReCiPe 2016 v1.03, midpoint (H)', 'material resources: metals/minerals', 'surplus ore potential (SOP)')","heat production, air-water heat pump 10kW_('ReCiPe 2016 v1.03, midpoint (H)', 'ozone depletion', 'ozone depletion potential (ODPinfinite)')","heat production, air-water heat pump 10kW_('ReCiPe 2016 v1.03, midpoint (H)', 'particulate matter formation', 'particulate matter formation potential (PMFP)')","heat production, air-water heat pump 10kW_('ReCiPe 2016 v1.03, midpoint (H)', 'photochemical oxidant formation: human health', 'photochemical oxidant formation potential: humans (HOFP)')","heat production, air-water heat pump 10kW_('ReCiPe 2016 v1.03, midpoint (H)', 'photochemical oxidant formation: terrestrial ecosystems', 'photochemical oxidant formation potential: ecosystems (EOFP)')","heat production, air-water heat pump 10kW_('ReCiPe 2016 v1.03, midpoint (H)', 'water use', 'water consumption potential (WCP)')"
Run ID,,,,,,,,,,,,,,,,,,,,,
0,1.860916e-09,1.584071e-13,5.799926e-09,-4.417642e-12,5.367560e-12,2.329998e-10,2.999347e-10,1.397100e-13,3.252541e-10,1.073142e-09,...,0.047749,0.583529,0.068903,0.002055,0.001046,4.043230e-08,0.000107,0.000127,0.000133,0.000855
1,1.654597e-09,1.539116e-13,5.635141e-09,3.590101e-11,1.322622e-11,2.949130e-10,4.221561e-10,1.358601e-13,3.173841e-10,1.006401e-09,...,0.003924,0.228749,0.043330,0.001364,0.000705,2.711935e-08,0.000074,0.000087,0.000089,0.000488
2,5.299540e-09,2.375466e-13,8.697154e-09,-1.327912e-10,-1.448851e-11,3.815466e-10,1.622667e-09,3.053832e-13,2.355601e-10,4.034417e-09,...,0.006245,0.225312,0.005523,0.000774,0.000718,3.470873e-08,0.000054,0.000074,0.000076,0.000372
3,1.333488e-09,1.517578e-13,5.557382e-09,4.316246e-11,1.184636e-11,1.380558e-10,4.934830e-10,1.343790e-13,4.011760e-10,9.414332e-10,...,0.010077,-0.344703,0.010820,0.001448,0.000783,2.902614e-08,0.000079,0.000107,0.000111,0.000542
4,1.302455e-09,1.585400e-13,5.806162e-09,9.232150e-11,2.326786e-11,2.192125e-10,3.277271e-10,1.247259e-13,6.090573e-10,6.684346e-10,...,0.006425,0.235820,0.106135,0.001867,0.000710,2.727544e-08,0.000073,0.000082,0.000085,0.000518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.000519e-09,1.378656e-13,5.047780e-09,2.224009e-10,4.501298e-11,1.5

In [ ]:
# df.to_csv('prerun_lca_MC10000.csv')